In [1]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from scipy import optimize
# import hvplot
import holoviews as hv
# import hvplot.pandas
from bokeh.plotting import figure, save, output_file
from bokeh.io import show
from bokeh.models import HoverTool
from bokeh.palettes import RdBu8 as Cividis8
from sklearn import metrics
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cdist
%matplotlib inline

# from google.colab import drive
# drive.mount('/content/drive')

In [93]:
# cv_file_list = []
path = 'E:'

export_path = 'C:/Users/****/pseudocapacitor/'
file_names = os.listdir(path)
file_names = sorted(file_names)


txt_file = []
for i in file_names:
    if i.split('.')[-1] == 'txt':
        txt_file.append(i)
file_names = txt_file
file_names = sorted(file_names)
print(len(file_names), "files are loaded")
#####################################################################
rates = []
voltage_list = np.linspace(0, 3, 1001)[np.newaxis, :]
r_cc_list = []
r_dc_list = []

df_cc_list = []
df_dc_list = []
cc_idx_list = []
dc_idx_list = []

rhs_list = []
rhs_sqrt_list = []
lhs_list_cc = []
lhs_list_dc = []
peak_x_dc = []
peak_y_dc =[]
peak_y_dc_sqrt = []
peak_x_cc = []
peak_y_cc =[]
for j in file_names:
    with open(os.path.join(path, j), 'r', encoding='utf-8') as f:
        lines = f.readlines()

        # extract rated
        no_h_line = int(lines[1].split()[-1])-1
        rate = float(lines[31].split()[-1])
        print(rate)
        rate_sqrt = np.sqrt(rate)
        rates.append(rate)

        # extract useful columns
        table = [s.split('\t') for s in lines[no_h_line:]]
        df = pd.DataFrame(table[1:], columns=table[0])
        df = df[['ox/red', 'Ewe/V', '<I>/mA', 'cycle number']].astype(float)
#         df = df.loc[df['cycle number'] == 1].drop(['cycle number'], axis=1)

        # split cc and dc from table
        df_cc = df.loc[df['ox/red'] == 0].drop(['ox/red'], axis=1) # ox/red=0
        df_dc = df.loc[df['ox/red'] == 1].drop(['ox/red'], axis=1) # ox/red=1
        
        
        #find the peak x and peak y 
        peak_p_dc = df_dc.loc[df_dc['<I>/mA'] == df_dc['<I>/mA'].max()]
        peak_x_dc.append(peak_p_dc['Ewe/V']) #this is V
        peak_y_dc_sqrt.append(peak_p_dc['<I>/mA']/rate_sqrt)  # this is the y for fitting
        peak_y_dc.append(peak_p_dc['<I>/mA']) # this is I 
#                                 
#         peak_p_cc = df_cc.loc[df_cc['Ewe/V'] >1] #here the value of 1 should be a variable
#         peak_p_cc = peak_p_cc.loc[peak_p_cc['<I>/mA'] == peak_p_cc['<I>/mA'].min()]
#         peak_x_cc.append(peak_p_cc['Ewe/V'])
#         peak_y_cc.append(peak_p_cc['<I>/mA']/rate_sqrt)
# end of finding peak
        
        df_cc = df_cc.sort_values(['Ewe/V']) # .reset_index()
        df_dc = df_dc.sort_values(['Ewe/V']) # .reset_index()
        df_cc_list.append(df_cc)
        df_dc_list.append(df_dc)     # data with dc data

        # extract cc rows that 'eve/v' close to voltage values
        voltage_matrix = np.tile(voltage_list, (df_cc.shape[0], 1))
        df_cc_x = df_cc['Ewe/V'].to_numpy()[:, np.newaxis] # extract 'Ewe/V' to ndarray
        diff = np.abs(voltage_matrix - df_cc_x) # difference to voltages
        cc_idx = np.argsort(diff, axis=0)[0, :] # extract index of min diff
        cc_y = df_cc['<I>/mA'].iloc[cc_idx] / rate_sqrt
        lhs_list_cc.append(cc_y)

        # extract dc rows that 'eve/v' close to voltage values
        voltage_matrix = np.tile(voltage_list, (df_dc.shape[0], 1))
        df_dc_x = df_dc['Ewe/V'].to_numpy()[:, np.newaxis] # extract 'Ewe/V' to ndarray
        diff = np.abs(voltage_matrix - df_dc_x) # difference to voltages
        dc_idx = np.argsort(diff, axis=0)[0, :] # extract index of min diff
        dc_y = df_dc['<I>/mA'].iloc[dc_idx] / rate_sqrt
        lhs_list_dc.append(dc_y)

        rhs_sqrt_list.append(rate_sqrt)   # this is sqrt of rate V
        rhs_list.append(rate)
        cc_idx_list.append(cc_idx)
        dc_idx_list.append(dc_idx)
#         z_cc = np.polyfit(rate_sqrt, lhs_list_cc,1)               
lhs_list_cc = np.array(lhs_list_cc).T
lhs_list_dc = np.array(lhs_list_dc).T
rhs_list = np.array(rhs_list)[:, np.newaxis]
rhs_sqrt_list = np.array(rhs_sqrt_list)[:, np.newaxis]
#####################################################################
#find the k1 and k2 for the peaks-line
plinear_regressor = LinearRegression()  
plinear_regressor.fit(rhs_sqrt_list, peak_y_dc_sqrt)
peak_k1 = plinear_regressor.coef_[0][0] # this is k1
peak_k2 = plinear_regressor.intercept_[0]  # this k2
print("peak line, k1 is: ", peak_k1, ", and k2 is: ", peak_k2)
#####################################################################
#find the linear function of peaks-line
pl_linear_regressor = LinearRegression()
pl_linear_regressor.fit(peak_x_dc, peak_y_dc)
pslope = pl_linear_regressor.coef_[0][0]
pitcp = pl_linear_regressor.intercept_[0]
peak_width = -peak_x_dc[0].to_numpy()[0] + peak_x_dc[-1].to_numpy()[0]
print("peak width is : ", peak_width)
round_width = round(peak_width, 2)
print("round peak width is : ", round_width)
#####################################################################
# find the parallel line of peak-line
#find a point at the lowest rate with the given x value, for example x =1
end_new_x = round(peak_x_dc[0],1).to_numpy()[0]
new_x_list = np.arange(0,end_new_x, 0.01)
new_x_k1_list = []
new_x_k2_list = []
k_x_list = [] # the x of filtered points with k value
for new_x in new_x_list:
#     print(new_x)
    result_index = df_dc_list[0]['Ewe/V'].sub(new_x).abs().idxmin()  # index of the intersection between new_x and the lowest
    intersection = df_dc_list[0].loc[result_index]  # the start point of the line at  x = new_x
    line_x = intersection['Ewe/V']
    line_y = intersection['<I>/mA']
    line_intercept = line_y - line_x*pslope
    xr = np.arange(new_x,new_x+round_width, 0.0001 ) # the x range of the parallel line
    line_function = xr*pslope+line_intercept  # the y range of the parallel line
    # plot dc and the new parallel line
    #####################################################################
#     for df_cc, df_dc in zip(df_cc_list, df_dc_list):
#     #     plt.plot(df_cc['Ewe/V'], df_cc['<I>/mA'])
#         plt.plot(df_dc['Ewe/V'], df_dc['<I>/mA'])
#     plt.scatter(peak_x_dc, peak_y_dc)  # peak points
#     plt.xlabel('Ewe/V')
#     plt.ylabel('<I>/mA')
#     plt.plot(xr, line_function)  # new parallel line
    # plt.xlim(0.998, 1.015)
    #####################################################################
    # find the other four intersections through comparing distance
    # new_points_list is the list of intersection points
    # to be noticed, the first element needs to be replaced

    new_points = np.array([xr, line_function]).T
    new_points_list = []
    for df_dc in df_dc_list:
        df_dc_x = (df_dc['Ewe/V'].loc[df_dc['Ewe/V'].between(new_x,new_x+round_width)]).to_numpy()
        df_dc_y = (df_dc['<I>/mA'].loc[df_dc['Ewe/V'].between(new_x,new_x+round_width)]).to_numpy()
#         plt.plot(df_dc_x, df_dc_y)
        new_df_dc = np.array([df_dc_x, df_dc_y]).T
        dist = cdist(new_points, new_df_dc)
        m = np.argmin(dist)
        r,c = divmod(m, dist.shape[1])
        new_points_list.append(new_df_dc[c])
#         plt.scatter(new_df_dc[c][0], new_df_dc[c][1], color = 'red')
        
    new_points_pos = np.array(new_points_list).T  
    new_points_y = new_points_pos[1]  # y value of the five intersection points
    new_lhs_list = []
    for rate_sqrt, i in zip(rhs_sqrt_list, new_points_y):
        new_lhs = i/rate_sqrt
        new_lhs_list.append(new_lhs)
    # print(new_lhs_list)
    # LR for the five points
    ulinear_regressor = LinearRegression()  
    ulinear_regressor.fit(rhs_sqrt_list, new_lhs_list)
    u_k1 = ulinear_regressor.coef_[0][0] # this is k1
    u_k2 = ulinear_regressor.intercept_[0]  # this k2
    if u_k1 >= 0 and u_k2 >=0:
        new_x_k1_list.append(u_k1)
        new_x_k2_list.append(u_k2)
        k_x_list.append(new_x)
#     print("parallel line, k1 is: ", u_k1, ", and k2 is: ", u_k2)
print("Done of finding parallel lines")
#####################################################################
# get the point of k1 and k2
for j in np.arange(0, len(rates), 1):
#     print(j)
    df_dc_list[j].plot(x = 'Ewe/V', y = '<I>/mA', color = "black")
    k1y = np.array(new_x_k1_list)*rates[j]
    k2y = np.array(new_x_k2_list)*rhs_sqrt_list[j]
    plt.plot(k_x_list, k1y,'r.-.')
    plt.plot(k_x_list, k2y,'b.-.')
    plt.title("rate = "+str(rates[j]))
    plt.axhline(0, color = 'black')
    plt.fill_between(k_x_list, k1y, 0, hatch = '/', color = 'red', alpha=0.2)
    plt.fill_between(k_x_list, k2y, 0, hatch = '\\' ,color = 'blue', alpha=0.2)
    plt.savefig(export_path+file_names[j][:-4]+'.jpg', dpi = 600)

6
